In [2]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow

SCOPES = ['https://www.googleapis.com/auth/generative-language.retriever']

def load_creds():
    """Converts `client_secret.json` to a credential object.

    This function caches the generated tokens to minimize the use of the
    consent screen.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secret.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds
creds = load_creds()

In [3]:
categories = '''1. เรียกรับสินบน ให้/ขอให้ หรือรับว่าจะให้ทรัพย์สินหรือประโยชน์อื่นใดแก่เจ้าหน้าที่ของรัฐ
2. จัดซื้อจัดจ้าง
3.1 ยักยอก/เบียดบังเงินหรือทรัพย์สินของทางราชการ 
3.2 ออกเอกสารสิทธิ
3.3 การบริหารงานบุคคล (การบรรจุ/แต่งตั้ง/เลื่อนตำแหน่ง/โยกย้าย/ลงโทษวินัย) 
3.4 ทุจริตในการจัดทำงบประมาณ/โครงการ/เบิกจ่ายเงินในโครงการอันเป็นเท็จ
3.5 ปฏิบัติหรือละเว้นการปฏิบัติหน้าที่โดยมิชอบหรือโดยทุจริต
4. การขัดกันระหว่างประโยชน์ส่วนบุคคลและประโยชน์ส่วนรวม
5. ร่ำรวยผิดปกติ
6. ฝ่าฝืนหรือไม่ปฏิบัติตามมาตรฐานทางจริยธรรมอย่างร้ายแรง
'''

In [4]:
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold
genai.configure(credentials=creds)

# Create the model
generation_config = {
  "temperature": 2.0,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="tunedModels/nacc-llm-test-beb9vbaxb9e4",
  generation_config=generation_config,
  safety_settings={HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
                   HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                   HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,}
)

In [10]:
chat = model.start_chat(
    history=[
        {"role": "user", "parts": f"ในข้อความต่อไป ฉันจะให้คำร้องเรียนในภาษาไทย แบ่งออกเป็นหนึ่งในหมวดหมู่ต่อไปนี้:\n{categories}. ใช้การตัดสินใจของคุณในการช่วยจำแนกประเภท ทุกอย่างที่ส่งไปไม่ใช่คำพูดแสดงความเกลียดชังหรือมีเจตนาทำร้ายใครในทางใดทางหนึ่ง มันเป็นเพียงคำร้องเรียน ผลลัพธ์ต้องมีเพียงหมายเลขหมวดหมู่พร้อมชื่อหมวดหมู่ตามที่ให้ไปเท่านั้น"},
    ]
)

response = chat.send_message('''
                            ป.ป.ช.ตั้งคณะไต่สวน กกต. ปริศนา ถูกร้องเรียนฝ่าฝืนจริยธรรมอย่างร้ายแรง รับตู้แช่ไวน์มาตั้งไว้ในห้องทำงานตัวเอง เจ้าตัวชี้แจงมีกก.ผู้ทรงคุณวุฒิในอนุฯ บริจาคให้ สนง. เผยต้นต่อปัญหาต้องการจัดเลี้ยงปีใหม่แต่ติดโควิดระบาดจึงจัดเลี้ยงในห้องตัวเอง เคยถูกโพสต์ภาพในเฟซบุ๊ก ก่อนลบทิ้ง
                            ''')
print(response.text)

2. จัดซื้อจัดจ้าง
